In [86]:
from os import path

# Baseline mods

In [127]:
CMD = (
    "CUDA_VISIBLE_DEVICES={} python -m torch.distributed.launch --nproc_per_node=1 "
    "--master_addr='127.0.0.{}' --master_port=2950{} ")
CMD_main = (
    "train_dist.py "
    "--model resnet50 --batch_size 128 --lr 1e-04 --test_batch_size 128 "
    "--no_use_several_test_samples --epochs 200 --use_last_n_layers {no_layers} "
    "--save_dir experiments/new_baseline_mods/r50_128_300_{tile_size}_{version}_{small}_{no_layers}_{no} "
    "--tile_size {tile_size} --data_version {version} --seed {seed}")

In [128]:
all_cmds = []
for no_layers in [-1, 9]:
    for s, seed in enumerate([10, 42, 100]):
        for tile_size in [300, 600, 1200]:
            for data_version in ["v1", "v2"]:
                for data_mod in ["", "osm_img slope waterways admin_bounds_qgis"]:
                    cmd = CMD_main.format(tile_size=tile_size, version=data_version, 
                                          small="small" if data_mod != "" else "large",
                                          seed=seed, no=s, no_layers=no_layers)
                    if data_mod != "":
                        cmd += " --data_modalities {}".format(data_mod)
                    all_cmds.append(cmd)

gpu_jobs = {i: "" for i in range(8)}
for i, cmd in enumerate(all_cmds):
    gpucmd = CMD.format((i//2) % 4, i % 8 + 1, i % 8 + 1) + cmd + "; "
    gpu_jobs[i % 8] += gpucmd
print(len(all_cmds))

72


In [129]:
gpu_jobs

{0: "CUDA_VISIBLE_DEVICES=0 python -m torch.distributed.launch --nproc_per_node=1 --master_addr='127.0.0.1' --master_port=29501 train_dist.py --model resnet50 --batch_size 128 --lr 1e-04 --test_batch_size 128 --no_use_several_test_samples --epochs 200 --use_last_n_layers -1 --save_dir experiments/new_baseline_mods/r50_128_300_300_v1_large_-1_0 --tile_size 300 --data_version v1 --seed 10; CUDA_VISIBLE_DEVICES=0 python -m torch.distributed.launch --nproc_per_node=1 --master_addr='127.0.0.1' --master_port=29501 train_dist.py --model resnet50 --batch_size 128 --lr 1e-04 --test_batch_size 128 --no_use_several_test_samples --epochs 200 --use_last_n_layers -1 --save_dir experiments/new_baseline_mods/r50_128_300_1200_v1_large_-1_0 --tile_size 1200 --data_version v1 --seed 10; CUDA_VISIBLE_DEVICES=0 python -m torch.distributed.launch --nproc_per_node=1 --master_addr='127.0.0.1' --master_port=29501 train_dist.py --model resnet50 --batch_size 128 --lr 1e-04 --test_batch_size 128 --no_use_several_

In [130]:
FP = "../experiments/new_baseline_mods/r50_128_300_{tile_size}_{version}_{small}_{no_layers}_{no}"
for no_layers in [-1, 9]:
    for s, seed in enumerate([10, 42, 100]):
        for tile_size in [300, 600, 1200]:
            for data_version in ["v1", "v2"]:
                for data_mod in ["", "osm_img slope waterways admin_bounds_qgis"]:
                    fp = FP.format(tile_size=tile_size, version=data_version, 
                                   small="small" if data_mod != "" else "large",
                                   seed=seed, no=s, no_layers=no_layers)
                    if not path.isdir(fp):
                        print("Path {} does not exist".format(fp))
                        continue
                    if not path.isfile(path.join(fp, "logs.json")):
                        print("Path {} does not have logs.json yet.".format(fp))

In [132]:
EVAL_CMD = "python train_dist.py --evaluate --save_dir {save_dir}"
FP = "./experiments/new_baseline_mods/r50_128_300_{tile_size}_{version}_{small}_{no_layers}_{no}"
all_cmds = []
for no_layers in [-1, 9]:
    for s, seed in enumerate([10, 42, 100]):
        for tile_size in [300, 600, 1200]:
            for data_version in ["v1", "v2"]:
                for data_mod in ["", "osm_img slope waterways admin_bounds_qgis"]:
                    for no_use_several_samples in [True, False]:
                        fp = FP.format(tile_size=tile_size, version=data_version, 
                                       small="small" if data_mod != "" else "large",
                                       seed=seed, no=s, no_layers=no_layers)
                        if no_use_several_samples:
                            fp += " --no_use_several_test_samples"
                        cmd = EVAL_CMD.format(save_dir=fp)
                        all_cmds.append(cmd)

CMD_GPU = "CUDA_VISIBLE_DEVICES={} "
gpu_jobs = {i: "" for i in range(4)}
for i, cmd in enumerate(all_cmds):
    gpucmd = CMD_GPU.format(i % 4) + cmd + "; "
    gpu_jobs[i % 4] += gpucmd
print(len(all_cmds))

144


In [133]:
gpu_jobs

{0: 'CUDA_VISIBLE_DEVICES=0 python train_dist.py --evaluate --save_dir ./experiments/new_baseline_mods/r50_128_300_300_v1_large_-1_0 --no_use_several_test_samples; CUDA_VISIBLE_DEVICES=0 python train_dist.py --evaluate --save_dir ./experiments/new_baseline_mods/r50_128_300_300_v2_large_-1_0 --no_use_several_test_samples; CUDA_VISIBLE_DEVICES=0 python train_dist.py --evaluate --save_dir ./experiments/new_baseline_mods/r50_128_300_600_v1_large_-1_0 --no_use_several_test_samples; CUDA_VISIBLE_DEVICES=0 python train_dist.py --evaluate --save_dir ./experiments/new_baseline_mods/r50_128_300_600_v2_large_-1_0 --no_use_several_test_samples; CUDA_VISIBLE_DEVICES=0 python train_dist.py --evaluate --save_dir ./experiments/new_baseline_mods/r50_128_300_1200_v1_large_-1_0 --no_use_several_test_samples; CUDA_VISIBLE_DEVICES=0 python train_dist.py --evaluate --save_dir ./experiments/new_baseline_mods/r50_128_300_1200_v2_large_-1_0 --no_use_several_test_samples; CUDA_VISIBLE_DEVICES=0 python train_dis

# Baseline rgb

In [116]:
datasets = [
    "v1_b2p_rgb_large_590_320_jpg",
    "v1_b2p_rgb_large_1150_600_jpg",
    "v1_b2p_rgb_large_2350_1200_jpg",
    "v2_b2p_rgb_large_590_320_jpg",
    "v2_b2p_rgb_large_1150_600_jpg",
    "v2_b2p_rgb_large_2350_1200_jpg"
]

CMD = "CUDA_VISIBLE_DEVICES={} "

CMD_main = (
    "python train_baseline.py "
    "--model resnet50 --batch_size 128 --lr 1e-04 "
    "--num_epochs 200 "
    "--output_dir experiments/new_baseline_rgb/r50_{tile_size}_{version}_{last_layer}_{no} "
    "--dataset_name {dataset_name} "
    "--use_last_n_layers {last_layer} --no_use_pretrained --seed {seed}")

In [117]:
cmds = []
for no, seed in enumerate([10, 42, 100]):
    for last_layer in [-1, 9]:
        for dataset in datasets:
            tile_size = dataset.split("_")[-2]
            version = dataset.split("_")[0]
            cmd = CMD_main.format(
                tile_size=tile_size, version=version, last_layer=last_layer, seed=seed,
                no=no, dataset_name=dataset)
            cmds.append(cmd)

gpu_jobs = {i: "" for i in range(8)}
for i, cmd in enumerate(cmds):
    gpucmd = CMD.format((i//2) % 4) + cmd + "; "
    gpu_jobs[i % 8] += gpucmd
print(len(cmds))

36


In [118]:
gpu_jobs

{0: 'CUDA_VISIBLE_DEVICES=0 python train_baseline.py --model resnet50 --batch_size 128 --lr 1e-04 --num_epochs 200 --output_dir experiments/new_baseline_rgb/r50_320_v1_-1_0 --dataset_name v1_b2p_rgb_large_590_320_jpg --use_last_n_layers -1 --no_use_pretrained --seed 10; CUDA_VISIBLE_DEVICES=0 python train_baseline.py --model resnet50 --batch_size 128 --lr 1e-04 --num_epochs 200 --output_dir experiments/new_baseline_rgb/r50_1200_v1_9_0 --dataset_name v1_b2p_rgb_large_2350_1200_jpg --use_last_n_layers 9 --no_use_pretrained --seed 10; CUDA_VISIBLE_DEVICES=0 python train_baseline.py --model resnet50 --batch_size 128 --lr 1e-04 --num_epochs 200 --output_dir experiments/new_baseline_rgb/r50_600_v2_-1_1 --dataset_name v2_b2p_rgb_large_1150_600_jpg --use_last_n_layers -1 --no_use_pretrained --seed 42; CUDA_VISIBLE_DEVICES=0 python train_baseline.py --model resnet50 --batch_size 128 --lr 1e-04 --num_epochs 200 --output_dir experiments/new_baseline_rgb/r50_320_v1_-1_2 --dataset_name v1_b2p_rgb_l